# 🚀 IESA DeepTech Hackathon 2026 - Quick Start

**Edge AI Defect Detection System**  
One-click training pipeline for Google Colab

---

## 📋 Prerequisites

1. Upload `Dataset.zip` to your Google Drive
2. Ensure dataset structure: `Dataset/{train,val,test}/{class_folders}`
3. Run cells sequentially

**Estimated Runtime**: 2-3 hours (with GPU)

## 1️⃣ Environment Setup

In [ ]:
# Check GPU availability
!nvidia-smi

# Enable GPU acceleration
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

In [ ]:
# Clone repository (replace with your GitHub URL)
!git clone https://github.com/yourusername/iesa-defect-detection.git
%cd iesa-defect-detection

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

print("✅ Installation complete!")

## 2️⃣ Mount Google Drive & Extract Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Verify dataset location
import os
dataset_zip = "/content/drive/MyDrive/Dataset.zip"

if os.path.exists(dataset_zip):
    print(f"✅ Dataset found: {dataset_zip}")
else:
    print("❌ Dataset.zip not found! Please upload to Google Drive.")

In [ ]:
# Extract dataset
import zipfile

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/')

print("✅ Dataset extracted to /content/Dataset")

# Verify structure
!ls -lh /content/Dataset

## 3️⃣ Run Full Training Pipeline

In [ ]:
# Run complete pipeline (wafer + die training)
!python iesa_defect_detection_pipeline.py

## 4️⃣ Evaluate Results

In [ ]:
# Load metrics
import json

with open('outputs/results/wafer_metrics.json', 'r') as f:
    wafer_metrics = json.load(f)

print("Wafer Model Performance:")
print(json.dumps(wafer_metrics['metrics'], indent=2))

In [ ]:
# Display confusion matrix
from IPython.display import Image, display

display(Image(filename='outputs/results/wafer_confusion_matrix.png'))
display(Image(filename='outputs/results/wafer_confidence_dist.png'))

## 5️⃣ Test Inference Pipeline

In [ ]:
# Import inference module
from iesa_defect_detection_pipeline import InferencePipeline

# Load models
pipeline = InferencePipeline(
    wafer_model_path="outputs/models/wafer_best.h5",
    die_model_path="outputs/models/die_best.h5"
)

print("✅ Inference pipeline ready!")

In [ ]:
# Test on sample image
import glob

# Get first test image
test_images = glob.glob('/content/Dataset/test/**/*.jpg', recursive=True)
sample_image = test_images[0]

# Run inference
result = pipeline.predict(sample_image, use_tiling=True, early_exit=True)

# Display results
print("\n" + "="*60)
print("INFERENCE RESULT")
print("="*60)
print(f"Image: {sample_image}")
print(f"Stage: {result['stage']}")
print(f"Predicted Class: {result['predicted_class']}")
print(f"Confidence: {result['confidence']:.2%}")
print(f"Early Exit: {result['early_exit']}")
print(f"Tiling Used: {result['tiling_used']}")
print("="*60)

# Show image
from PIL import Image
display(Image.open(sample_image).resize((400, 400)))

## 6️⃣ Download Outputs

In [ ]:
# Zip all outputs
!zip -r outputs.zip outputs/

# Download to local machine
from google.colab import files
files.download('outputs.zip')

print("✅ Download started! Check your browser's download folder.")

## 7️⃣ Export ONNX Models

In [ ]:
# ONNX models should be already exported by the pipeline
# Verify they exist
import os

onnx_files = [
    'outputs/models/wafer_model.onnx',
    'outputs/models/die_model.onnx'
]

for file in onnx_files:
    if os.path.exists(file):
        size_mb = os.path.getsize(file) / (1024 * 1024)
        print(f"✅ {file} ({size_mb:.2f} MB)")
    else:
        print(f"❌ {file} not found!")

## 🎯 Next Steps

1. **Review Metrics**: Check `outputs/results/` for detailed performance reports
2. **Test Edge Deployment**: Use `wafer_model.onnx` and `die_model.onnx` with NXP eIQ
3. **Iterate**: Adjust hyperparameters in `Config` class for better performance
4. **Create Submission**: Package code, models, and report for hackathon

---

**Questions?** Check README.md for detailed documentation.